<a href="https://colab.research.google.com/github/riyasachdeva04/MLOps/blob/main/zenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

#automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
⠏ Installing integrations...


{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN = "2TqvydXoY8EoYMKI09qUA5tg25Q_4cfcyJgGkkpF9c9kMM5rK"

In [3]:
from zenml.environment import Environment

if Environment.in_google_colab():

  # auth ngrok
  !pip install ngrok
  !ngrok authtoken {NGROK_TOKEN}

/bin/bash: line 1: ngrok: command not found


In [4]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
ZenML repository initialized at /content.
⠏ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration 
will only take effect when you're running ZenML from the initialized repository 
root, or from a subdirectory. For more information on repositories and 
configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


In [5]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
  model = SVC(gamma = 0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test_ACcuracy: {test_acc}")

train_test()

Test_ACcuracy: 0.9583333333333334


In [7]:
# building pipeline
#steps: data loading, model training, model evaluation

from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:

  # load digits dataset as np array
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size=0.2, shuffle=False
  )
  return X_train, X_test, y_train, y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
# train sklearn SVC classifier
  model = SVC(gamma = 0.001)
  model.fit(X_train, y_train)
  return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
  # calculate accuracy
  test_accuracy = model.score(X_test, y_test)
  print(test_accuracy)
  return test_accuracy

In [8]:
# connecting all steps into an ML pipeline

from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train = X_train, y_train = y_train)
  eval = evaluator(X_test = X_test, y_test = y_test, model = model)


In [12]:
# running zenML pipeline
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered version: (version: 1).
Executing a new run.
Caching disabled.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Run digits_pipeline-2023_08_11-21_43_05_864963 has finished in 0.910s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [18]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=65ea23ec54704f46ab00488c79dbf74fffb512c6e6bdd99eeda619ab0ef6bb53
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [19]:
from zenml.environment import Environment

def start_zenml_dashboard(port = 8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-4f8478f6-b14f-495f-9b40-a30085414c93


Opening tunnel named: http-8237-4f8478f6-b14f-495f-9b40-a30085414c93


INFO:pyngrok.process.ngrok:t=2023-08-11T21:50:36+0000 lvl=info msg="no configuration paths supplied"


t=2023-08-11T21:50:36+0000 lvl=info msg="no configuration paths supplied"


t=2023-08-11T21:50:36+0000 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=/root/.config/ngrok/ngrok.yml legacy_path=/root/.ngrok2/ngrok.yml


...

t=2023-08-11T21:50:36+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-4f8478f6-b14f-495f-9b40-a30085414c93 addr=http://localhost:8237 url=https://b45b-34-73-115-85.ngrok.io


INFO:pyngrok.process.ngrok:t=2023-08-11T21:50:36+0000 lvl=info msg=end pg=/api/tunnels id=0f85193181a6de1f status=201 dur=36.489761ms


t=2023-08-11T21:50:36+0000 lvl=info msg=end pg=/api/tunnels id=0f85193181a6de1f status=201 dur=36.489761ms
NumExpr defaulting to 2 threads.
Updating the local ZenML server.
⠋ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠙ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠹ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠸ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠼ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠴ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠦ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4b-e21beabd94ee] (type: 
zen_server, flavor: local)'.
⠧ Stopping service 'LocalZenServer[1d098cc5-bf4a-44cc-ab4

INFO:pyngrok.process.ngrok:t=2023-08-11T21:53:47+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2023-08-11T21:53:47+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
Shutting down
INFO:     Waiting for application shutdown.
Waiting for application shutdown.
INFO:     Application shutdown complete.
Application shutdown complete.
INFO:     Finished server process [31228]
Finished server process [31228]
